# Data Wrangling with DataFrames Coding Quiz

Use this Jupyter notebook to find the answers to the quiz in the previous section. There is an answer key in the next part of the lesson.

In [1]:
from pyspark.sql import SparkSession

# TODOS: 
# 1) import any other libraries you might need
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType
from pyspark.sql import Window
# 2) instantiate a Spark session 
spark = SparkSession.builder \
    .master("local") \
    .appName("Word Count") \
    .getOrCreate()
        
# 3) read in the data set located at the path "data/sparkify_log_small.json"
df = spark.read.json("data/sparkify_log_small.json")
# 4) write code to answer the quiz questions 

# Question 1

Which page did user id "" (empty string) NOT visit?

In [2]:
# TODO: write your code to answer question 1
print(df.take(5))
print(df.printSchema())
print(df.select("Page").where("userId = ''").distinct().collect())

[Row(artist='Showaddywaddy', auth='Logged In', firstName='Kenneth', gender='M', itemInSession=112, lastName='Matthews', length=232.93342, level='paid', location='Charlotte-Concord-Gastonia, NC-SC', method='PUT', page='NextSong', registration=1509380319284, sessionId=5132, song='Christmas Tears Will Fall', status=200, ts=1513720872284, userAgent='"Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"', userId='1046'), Row(artist='Lily Allen', auth='Logged In', firstName='Elizabeth', gender='F', itemInSession=7, lastName='Chase', length=195.23873, level='free', location='Shreveport-Bossier City, LA', method='PUT', page='NextSong', registration=1512718541284, sessionId=5027, song='Cheryl Tweedy', status=200, ts=1513720878284, userAgent='"Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"', userId='1000'), Row(artist='Cobra Starship Featuring Leighton Meester', auth='Logged In', firstName

# Question 2 - Reflect

What type of user does the empty string user id most likely refer to?


In [3]:
# TODO: use this space to explore the behavior of the user with an empty string
empty_users = df.filter(df.userId == "")
print(empty_users.select("artist").describe().show())
print(empty_users.select("auth").describe().show())
print(empty_users.select(["gender", "firstName", "lastName", "length", "location"]).describe().show())
print(empty_users.select(["method", "page", "registration", "sessionId", "userAgent", "status"]).describe().show())
print(empty_users.select(["ts", "level", "song"]).describe().show())
print(empty_users.groupBy(["page", "method", "auth"]).count().show())
#Users who are only visiting the About, Home and Login pages are probably unregistered visitors.

+-------+------+
|summary|artist|
+-------+------+
|  count|     0|
|   mean|  null|
| stddev|  null|
|    min|  null|
|    max|  null|
+-------+------+

None
+-------+----------+
|summary|      auth|
+-------+----------+
|  count|       336|
|   mean|      null|
| stddev|      null|
|    min|     Guest|
|    max|Logged Out|
+-------+----------+

None
+-------+------+---------+--------+------+--------+
|summary|gender|firstName|lastName|length|location|
+-------+------+---------+--------+------+--------+
|  count|     0|        0|       0|     0|       0|
|   mean|  null|     null|    null|  null|    null|
| stddev|  null|     null|    null|  null|    null|
|    min|  null|     null|    null|  null|    null|
|    max|  null|     null|    null|  null|    null|
+-------+------+---------+--------+------+--------+

None
+-------+------+-----+------------+------------------+---------+-----------------+
|summary|method| page|registration|         sessionId|userAgent|           status|
+-----

# Question 3

How many female users do we have in the data set?

In [4]:
# TODO: write your code to answer question 3
print(df.drop_duplicates(subset = ["userId"]).where("gender = 'F'").count())
# There are 462 female users in total

462


# Question 4

How many songs were played from the most played artist?

In [5]:
# TODO: write your code to answer question 4
df.groupBy("artist").count().orderBy('count', ascending = False).show()
# There were 83 songs were played for the most played artist (excluding NULL)

+--------------------+-----+
|              artist|count|
+--------------------+-----+
|                null| 1653|
|            Coldplay|   83|
|       Kings Of Leon|   69|
|Florence + The Ma...|   52|
|            BjÃÂ¶rk|   46|
|       Dwight Yoakam|   45|
|       Justin Bieber|   43|
|      The Black Keys|   40|
|         OneRepublic|   37|
|                Muse|   36|
|        Jack Johnson|   36|
|           Radiohead|   31|
|        Taylor Swift|   29|
|Barry Tuckwell/Ac...|   28|
|          Lily Allen|   28|
|               Train|   28|
|           Daft Punk|   27|
|           Metallica|   27|
|          Nickelback|   27|
|          Kanye West|   26|
+--------------------+-----+
only showing top 20 rows



# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.



In [6]:
# TODO: write your code to answer question 5
function = F.udf(lambda ishome : int(ishome == 'Home'), IntegerType())

user_window = Window \
    .partitionBy('userID') \
    .orderBy(F.desc('ts')) \
    .rangeBetween(Window.unboundedPreceding, 0)

cusum = df.filter((df.page == 'NextSong') | (df.page == 'Home')) \
    .select('userID', 'page', 'ts') \
    .withColumn('homevisit', function(F.col('page'))) \
    .withColumn('period', F.sum('homevisit').over(user_window))

cusum.filter((cusum.page == 'NextSong')) \
    .groupBy('userID', 'period') \
    .agg({'period':'count'}) \
    .agg({'count(period)':'avg'}).show()

+------------------+
|avg(count(period))|
+------------------+
| 6.898347107438017|
+------------------+

